In [1]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump

In [2]:
base_path = "/home/docker_opr/Datasets/NCLT_preprocessed"
train_df_path = os.path.join(base_path, "train.csv")

train_df = pd.read_csv(train_df_path)
train_df.head(2)

,Unnamed: 0,track,image,pointcloud,northing,easting
0,0,2012-01-08,1326030979526128,1326030979526128,0.338313,0.222726
1,1,2012-01-08,1326031015326922,1326031015326922,-2.798348,-9.238316


In [3]:
tracks = train_df["track"].unique()

descriptions = []

for track in tracks:
    cur_track_path = os.path.join(base_path, track)
    cur_track_df = train_df[train_df["track"] == track]
    images = cur_track_df.image.values
    images = [str(int(i))+".png" for i in images]
    for cam_id in range(1, 6):
        cam_df_path = os.path.join(cur_track_path, f"descriptions_Cam{cam_id}.csv")
        cam_df = pd.read_csv(cam_df_path)
        cam_descriptions = cam_df[cam_df["path"].isin(images)]["description"].values
        descriptions.append(cam_descriptions)
        
descriptions = np.hstack((descriptions))

In [4]:
descriptions.shape, len(train_df) * 5

((14640,), 14640)

In [5]:
def train_tfidf_pca(corpus, 
                    max_features_tfidf=None,
                    n_components_pca=100, 
                    base_savepath="./opr/datasets/"):
    vectorizer = TfidfVectorizer(max_features=max_features_tfidf)
    vectorizer.fit(corpus)
    print("n tfidf features = ", vectorizer.get_feature_names_out().shape)
    vectorized_corpus = vectorizer.transform(corpus).toarray()

    pca = PCA(n_components=n_components_pca)
    pca.fit(vectorized_corpus)
    
    vectorizer_savepath = os.path.join(base_savepath, 'vectorizer.joblib')
    pca_savepath = os.path.join(base_savepath, 'pca.joblib')
    dump(vectorizer, vectorizer_savepath) 
    dump(pca, pca_savepath) 

def text_transform(self, text):
    vect_data = self.vectorizer.transform([text]).toarray()
    pca_data = self.pca.transform(vect_data)
    pca_data = torch.tensor(pca_data, dtype=torch.float32)
    return pca_data

In [6]:
train_tfidf_pca(descriptions)

n tfidf features =  (7857,)
